In [6]:
import sys
!{sys.executable} -m pip install psycopg2-binary
import psycopg2
import pandas as pd
import getpass
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('/Users/jessie/Desktop/preprocessed.csv', index_col=0)
df

,airline,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,date,DOW
206668,Air_India,Delhi,Evening,one,Night,Mumbai,Business,24.75,1,42220,2022-02-11,4
206669,Air_India,Delhi,Night,one,Night,Mumbai,Business,26.50,1,44450,2022-02-11,4
206670,Air_India,Delhi,Evening,one,Night,Mumbai,Business,6.67,1,46690,2022-02-11,4
206671,Vistara,Delhi,Evening,zero,Night,Mumbai,Business,2.17,1,50264,2022-02-11,4
206672,Air_India,Delhi,Night,one,Afternoon,Mumbai,Business,17.75,1,50669,2022-02-11,4
...,...,...,...,...,...,...,...,...,...,...,...,...
206661,Vistara,Chennai,Early_Morning,one,Night,Hyderabad,Economy,13.83,49,7697,2022-03-31,3
206662,Vistara,Chennai,Early_Morning,one,Night,Hyderabad,Economy,13.83,49,7709,2022-03-31,3
206663,Vistara,Chennai,Afternoon,one,Morning,Hyderabad,Economy,20.58,49,8640,2022-03-31,3
206664,Vistara,Chennai,Morning,one,Morning,Hyderabad,Economy,23.33,49,8640,2022-03-31,3


In [3]:
from sklearn.preprocessing import LabelEncoder
df['date']=pd.to_datetime(df['date'])

columns_to_encode = ['airline','source_city','destination_city','class']

# Create a LabelEncoder object
le = LabelEncoder()

# Loop over the columns and apply the LabelEncoder
for col in columns_to_encode:
    df[col] = le.fit_transform(df[col])

In [8]:
import statsmodels.api as sm
from pmdarima import auto_arima
df_grouped = df.groupby(['airline', 'source_city', 'destination_city','class', 'date'])['price'].mean().reset_index()
results_df = pd.DataFrame(columns=['date', 'actual', 'predicted'])
for i, group in df_grouped.groupby(['airline', 'source_city', 'destination_city','class']):
    # Create a time series for this combination of origin and destination cities and cabin class
    ts = group.set_index('date')
    train= ts.loc['2022-02-11':'2022-03-16'] 
    test = ts.loc['2022-03-17':]

    stepwise_model = auto_arima(train[['price']], exogenous=train[['airline', 'source_city', 'destination_city','class']], start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=0, D=0, trace=False,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

    # Fit the model
    stepwise_model.fit(train[['price']], exogenous=train[['airline', 'source_city', 'destination_city','class']])

    test_predictions = stepwise_model.predict(n_periods=len(test), exogenous=test[['airline', 'source_city', 'destination_city','class']])
    
    # Create temporary DataFrame for test results
    temp_df = pd.DataFrame({'date': test.index, 'actual': np.array(test['price']), 'predicted': test_predictions})
    
    # Append to results DataFrame
    results_df = pd.concat([results_df, temp_df], ignore_index=True)

In [9]:
results_df

,date,actual,predicted
0,2022-03-17 00:00:00,59877.727273,59051.071614
1,2022-03-18 00:00:00,55372.384615,54935.861348
2,2022-03-19 00:00:00,57859.9375,56525.029183
3,2022-03-20 00:00:00,58483.461538,57199.889258
4,2022-03-21 00:00:00,55372.384615,54689.54102
...,...,...,...
2235,2022-03-27 00:00:00,6235.938776,7353.751706
2236,2022-03-28 00:00:00,6268.34,7425.522851
2237,2022-03-29 00:00:00,6247.34,7493.030137
2238,2022-03-30 00:00:00,6180.632653,7556.526851


In [10]:
# testing results
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE

rmse = np.sqrt(MSE(results_df['actual'],results_df['predicted']))
r2 = r2_score(results_df['actual'],results_df['predicted'])

results_df ['residuals']=results_df.actual-results_df.predicted
results_df['MAPE'] = np.abs((results_df['actual'] - results_df['predicted']) / results_df['actual'])

print ('The testing R^2 is: ', r2)
print ('The testing MAPE is: ', results_df.MAPE.mean())
print ('The testing RMSE is:', rmse)

The testing R^2 is:  0.9974033388571073
The testing MAPE is:  0.11505307841643592
The testing RMSE is: 1181.7508273471472
